# Data generation from spotifies API

In [1]:
import pandas as pd #Dataframe, Series
import numpy as np
import seaborn as sns
import graphviz
import io
from scipy import misc
from sklearn.metrics import accuracy_score
import spotipy
import spotipy.util as util
sp = spotipy.Spotify() 
from spotipy.oauth2 import SpotifyClientCredentials 
import spotipy.util as util
import pprint
import sys
from funcy import project
import time
%matplotlib inline 

In [ ]:
import json
with open('pass.json','r') as f:
    keys = json.load(f)['spotify_api']
    

In [1]:
#calling api
def refresh_api():
    scope = keys['scope']
    cid =keys['cid'] 
    secret = keys['secret']
    username = keys['username']
    client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
    sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    token=util.prompt_for_user_token(username,scope,client_id=cid,client_secret=secret,redirect_uri='https://google.com/')
    return sp
sp =refresh_api()

In [3]:
def song_handler(tracks,lists=True):
    
    if lists:
        features= sp.audio_features(tracks=tracks)
    else:
        features= sp.audio_features(tracks=[tracks]) #if you want for a song

    feat_list=['danceability','acousticness','energy','instrumentalness','key','liveness','loudness',
              'mode','tempo','valence']

    #Creating a dict with the featrues which will be later on to the playlist_df
    dictobj = project(features[0], feat_list)
    
    return dictobj
  
def playlist_handler(id):
    playlist = sp.user_playlist('spotify', playlist_id=id)
    songs=[]
    for tracks in playlist['tracks']['items']: 
        track = tracks['track']['id']
        songs.append(track)

    features = song_handler(songs)
    return features
#testing the function
dict1= playlist_handler('37i9dQZF1DXcBWIGoYBM5M')


In [5]:
def create_dataset(offsets,songs=False,earlystop=True): #if songs it creates a dataframe with all songs instead
    """Function that calls the api, and gets all the specific predefined """
    #song specific
    
    song_feature_list = ['danceability',
        'acousticness',
        'energy',
        'instrumentalness',
        'key',
        'liveness',
        'loudness',
        'mode',
        'tempo',
        'valence']   
    
    feats = {'name':[],
             'id':[],
             'popularity':[],
             'danceability':[],
        'acousticness':[],
        'energy':[],
        'instrumentalness':[],
        'key':[],
        'liveness':[],
        'loudness':[],
        'mode':[],
        'tempo':[],
        'valence':[]}
    
    if songs:
        feats['artists']=[]
        feats['album']=[]
        feats['explicit']=[]
    
    else:
        feats['followers']=[]
        feats['tot_tracks']=[]
        
    count = 0
    ii=0
    playlist_count=0
    playlists = sp.user_playlists('spotify', limit=50, offset=offsets)
    
    while playlists:
        for i, playlist in enumerate(playlists['items']):
            try:
                if earlystop and i>=250:
                    ii = 250
                    break
                
                metadata = sp.user_playlist('spotify', playlist_id=playlist['id'])                
                
                if songs:
                    if count%1000==0:
                        print(f"count: {count}")
                    for index, item in enumerate(metadata['tracks']['items']):
                        #get the song features
                        features= song_handler(item['track']['uri'],lists=False)
                        for k in song_feature_list:
                            feats[k].append(features[k])
                            
                        feats['name'].append(item['track']['name'])
                        feats['popularity'].append(item['track']['popularity'])
                        feats['explicit'].append(item['track']['explicit'])
                        feats['album'].append(item['track']['album']['name'])
                        feats['artists'].append(item['track']['artists'][0]['name'])
                        feats['id'].append(item['track']['uri'])
                        
                        
                        count+=1
                        if (len(feats['name'])!= len(feats['key'])):
                            print(f"lists not same lenghts for {feats['name']}")
                        
                        
                else: #for playlists
                    songs=[]
                    popularities = np.empty(len(metadata['tracks']['items']))
                    for index, item in enumerate(metadata['tracks']['items']): 
                        popularities[index] = item['track']['popularity']
                        songs.append(item['track']['id'])
                    #playlist specific appends
                    feats['popularity'].append(popularities.mean())
                    feats['name'].append(playlist['name'])
                    feats['id'].append(metadata['id'])
                    feats['followers'].append(metadata['followers']['total'])    
                    feats['tot_tracks'].append(playlist['tracks']['total'])
                    
                    features= song_handler(songs,lists=True) #getting the audio features
                    for k in feats:
                        feats[k].append(features[k])
                    
                print("%4d %s" % (i + 1 + playlists['offset'], playlist['name']))
                
                          
            except:
                print("NO METADATA")
        
        if earlystop and ii==250:break
        
        if playlists['next']:
            playlists = sp.next(playlists)
        else:
            playlists = None
            break
    
    return feats

#datadict= create_dataset(songs=True,earlystop=True)

In [ ]:
if songs: #Since we don't have artists for playlists
        feats['artists']=artists
        feats['album']=album
        feats['explicit']=explicit
    else:
        feats['followers']=followers
        feats['tot_tracks']=total_tracks
    
    feats['name']=names   
    feats['id']=ids 
    feats['popularity']=popularity_means
        

In [8]:
import json
def export_json(data,name="songs_2.json"):
    file = json.dumps(data)
    f = open(name,"w")
    f.write(file)
    f.close()
export_json(song_data,)

In [6]:

offset = [50,300,550,700,950,1200] #what index to return 
for i in offset:
    start= time.time()
    song_data= create_dataset(i,songs=True,earlystop=True)
    export_json(song_data,f"sample_songs{i}.json")
    df_songs =song_df(song_data,export=True,name=f"songs_data{i}.csv")
    print(f"Time: {time.time()-start}")

count: 0
  51 This Is Green Day
  52 This Is Metallica
  53 This Is Jane's Addiction
  54 This Is Oasis
retrying ...1secs
retrying ...1secs
  55 This Is KISS
retrying ...1secs
retrying ...1secs
  56 Dirty Rock
  57 One More Rep
retrying ...1secs
retrying ...1secs
  58 90s Rock Anthems
  59 Soak Up the Sun
NO METADATA
NO METADATA
  62 Rock Ballads
retrying ...1secs
retrying ...1secs
  63 Totally Alt
  64 Nu Metal Generation
  65 Alternative Beats
retrying ...1secs
retrying ...1secs
  66 Rock Covers
  67 Shoegaze & Beyond
retrying ...1secs
retrying ...1secs
  68 Rock Me UP!
  69 All Things Post
NO METADATA
  71 Classic Hardcore
  72 Rock Gaming
NO METADATA
NO METADATA
  75 Country Gold
  76 Texas Music Now
  77 Country Coffeehouse
  78 Breakout Country
  79 Country Kind of Love
retrying ...1secs
  80 Country Nights
NO METADATA
NO METADATA
  83 This Is: Merle Haggard
  84 This Is:  Johnny Cash
retrying ...1secs
retrying ...1secs
  85 From Beginning To End:  Johnny Cash
  86 90's Country
 

NameError: name 'export_json' is not defined

In [11]:
df_songs =song_df(song_data,export=True,name=f"songs_data2.csv")

In [10]:
def song_df(feats,export=False,name='songs_data.csv'):
    df = pd.DataFrame.from_dict(feats)
    
    df = df[['name','popularity', 'danceability', 'acousticness',
     'energy','instrumentalness','key','liveness','loudness','mode','tempo',
     'valence','artists','album','explicit','id']]
    if export:
        df.to_csv(name)
    #print(df.memory_usage())   
    return df
df_songs =song_df(song_data,export=True)
df_songs.head()


,name,popularity,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence,artists,album,explicit,id
0,Boulevard of Broken Dreams,72,0.490,0.003940,0.679,0.000015,8,0.0383,-3.680,1,167.046,0.506,Green Day,Boulevard of Broken Dreams,True,spotify:track:1hwJKpe0BPUsq6UUrwBWTw
1,Holiday - Faded Ending,66,0.538,0.000138,0.884,0.000000,8,0.4660,-3.251,1,146.511,0.736,Green Day,Holiday,False,spotify:track:5vfjUAhefN7IjHbTvVCT4Z
2,21 Guns,76,0.268,0.051800,0.742,0.000000,5,0.6260,-4.939,1,159.779,0.416,Green Day,21st Century Breakdown,False,spotify:track:64yrDBpcdwEdNY9loyEGbX
3,Basket Case,80,0.442,0.002930,0.943,0.000009,3,0.0910,-3.205,1,85.064,0.781,Green Day,Dookie,False,spotify:track:6L89mwZXSOwYl76YXfX13s
4,American Idiot,79,0.380,0.000026,0.988,0.000079,1,0.3680,-2.042,1,186.113,0.769,Green Day,American Idiot,True,spotify:track:6nTiIhLmQ3FWhvrGafw2zj


In [41]:
def playlist_df(feats,export=False,name='playlist_data.csv'):
    df= pd.DataFrame.from_dict(feats)
    df = df[['names', 'tot_tracks', 'popularity', 'danceability', 'acousticness',
         'energy','instrumentalness','key','liveness','loudness','mode','tempo',
         'valence','followers','popularity']]
    if export:
        df.to_csv(name)
    #print(df.memory_usage())
    return df

In [43]:
df.head()

,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence,names,tot_tracks,id,followers,popularity
0,0.794,0.0125,0.793,0.000000,11,0.0952,-4.521,0,123.941,0.677,Today's Top Hits,50,37i9dQZF1DXcBWIGoYBM5M,25162839,87.280000
1,0.803,0.2950,0.715,0.000134,2,0.0574,-3.280,1,101.085,0.574,¡Viva Latino!,50,37i9dQZF1DX10zKzsJ2jva,10449984,81.260000
2,0.621,0.0276,0.484,0.000001,2,0.1110,-7.033,0,76.532,0.159,New Music Friday,50,37i9dQZF1DX4JAvHpjipBk,3476967,24.520000
3,0.422,0.0176,0.711,0.000000,2,0.5590,-4.809,0,128.066,0.321,mint,64,37i9dQZF1DX4dyzvuaRJ0n,5570887,60.765625
4,0.620,0.0607,0.613,0.000000,0,0.1170,-5.830,1,82.430,0.483,Are & Be,39,37i9dQZF1DX4SBhb3fqCJd,5143572,69.846154


In [44]:
df = df[['names', 'tot_tracks', 'popularity', 'danceability', 'acousticness',
         'energy','instrumentalness','key','liveness','loudness','mode','tempo',
         'valence','followers','popularity']]
df.head()

,names,tot_tracks,popularity,danceability,acousticness,energy,instrumentalness,key,liveness,loudness,mode,tempo,valence,followers,popularity
0,Today's Top Hits,50,87.280000,0.794,0.0125,0.793,0.000000,11,0.0952,-4.521,0,123.941,0.677,25162839,87.280000
1,¡Viva Latino!,50,81.260000,0.803,0.2950,0.715,0.000134,2,0.0574,-3.280,1,101.085,0.574,10449984,81.260000
2,New Music Friday,50,24.520000,0.621,0.0276,0.484,0.000001,2,0.1110,-7.033,0,76.532,0.159,3476967,24.520000
3,mint,64,60.765625,0.422,0.0176,0.711,0.000000,2,0.5590,-4.809,0,128.066,0.321,5570887,60.765625
4,Are & Be,39,69.846154,0.620,0.0607,0.613,0.000000,0,0.1170,-5.830,1,82.430,0.483,5143572,69.846154


In [45]:
#Export
df.to_csv('playlists_upt.csv')